In [1]:
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

In [2]:
df = pd.read_csv('/home/harshal/Downloads/pokemon_alopez247.csv')
df.columns

Index(['Number', 'Name', 'Type_1', 'Type_2', 'Total', 'HP', 'Attack',
       'Defense', 'Sp_Atk', 'Sp_Def', 'Speed', 'Generation', 'isLegendary',
       'Color', 'hasGender', 'Pr_Male', 'Egg_Group_1', 'Egg_Group_2',
       'hasMegaEvolution', 'Height_m', 'Weight_kg', 'Catch_Rate',
       'Body_Style'],
      dtype='object')

In [3]:
df = df[['isLegendary','Generation', 'Type_1', 'Type_2', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed','Color','Egg_Group_1','Height_m','Weight_kg','Body_Style']]

In [4]:
df['isLegendary'] = df['isLegendary'].astype(int)

In [5]:
def dummy_creations(df, dummy_index):
    for i in dummy_index:
        df_dummy = pd.get_dummies(df[i])
        df = pd.concat([df, df_dummy], axis=1)
        df = df.drop(i, axis=1)
    return df

In [6]:
df = dummy_creations(df, ['Egg_Group_1', 'Body_Style', 'Color','Type_1', 'Type_2'])

In [7]:
def train_test_splitter(DataFrame, column):
    df_train = DataFrame.loc[df[column] != 1]
    df_test = DataFrame.loc[df[column] == 1]
    
    df_train = df_train.drop(column, axis=1)
    df_test = df_test.drop(column, axis=1)
    
    return df_train, df_test

df_train, df_test = train_test_splitter(df, 'Generation')

In [8]:
def label_delineator(df_train, df_test, label):
    
    train_data = df_train.drop(label, axis=1).values
    train_labels = df_train[label].values
    test_data = df_test.drop(label, axis=1).values
    test_labels = df_test[label].values
    return (train_data, train_labels, test_data, test_labels)

train_data, train_labels, test_data, test_labels = label_delineator(df_train, df_test, 'isLegendary')

In [9]:
def data_normalizer(train_data, test_data):
    train_Data = preprocessing.MinMaxScaler().fit_transform(train_data)
    test_data = preprocessing.MinMaxScaler().fit_transform(test_data)
    return (train_Data, test_data)

train_data, test_data = data_normalizer(train_data, test_data)

In [10]:
length = train_data.shape[1]

model = keras.Sequential()
model.add(keras.layers.Dense(500, activation='relu', input_shape=[length,]))
model.add(keras.layers.Dense(2, activation='softmax'))

In [11]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_data, train_labels, epochs=400)

Train on 570 samples
Epoch 1/400
570/570 [==============================] - 0s 526us/sample - loss: 0.5890 - accuracy: 0.8561
Epoch 2/400
570/570 [==============================] - 0s 168us/sample - loss: 0.4503 - accuracy: 0.9263
Epoch 3/400
570/570 [==============================] - 0s 138us/sample - loss: 0.3789 - accuracy: 0.9263
Epoch 4/400
570/570 [==============================] - 0s 118us/sample - loss: 0.3389 - accuracy: 0.9263
Epoch 5/400
570/570 [==============================] - 0s 131us/sample - loss: 0.3140 - accuracy: 0.9263
Epoch 6/400
570/570 [==============================] - 0s 140us/sample - loss: 0.2975 - accuracy: 0.9263
Epoch 7/400
570/570 [==============================] - 0s 197us/sample - loss: 0.2857 - accuracy: 0.9263
Epoch 8/400
570/570 [==============================] - 0s 221us/sample - loss: 0.2767 - accuracy: 0.9263
Epoch 9/400
570/570 [==============================] - 0s 147us/sample - loss: 0.2695 - accuracy: 0.9263
Epoch 10/400
570/570 [============

570/570 [==============================] - 0s 303us/sample - loss: 0.0842 - accuracy: 0.9877
Epoch 79/400
570/570 [==============================] - 0s 184us/sample - loss: 0.0831 - accuracy: 0.9877
Epoch 80/400
570/570 [==============================] - 0s 149us/sample - loss: 0.0820 - accuracy: 0.9877
Epoch 81/400
570/570 [==============================] - 0s 153us/sample - loss: 0.0810 - accuracy: 0.9842
Epoch 82/400
570/570 [==============================] - 0s 271us/sample - loss: 0.0800 - accuracy: 0.9877
Epoch 83/400
570/570 [==============================] - 0s 164us/sample - loss: 0.0791 - accuracy: 0.9877
Epoch 84/400
570/570 [==============================] - 0s 88us/sample - loss: 0.0781 - accuracy: 0.9877
Epoch 85/400
570/570 [==============================] - 0s 113us/sample - loss: 0.0772 - accuracy: 0.9877
Epoch 86/400
570/570 [==============================] - 0s 158us/sample - loss: 0.0764 - accuracy: 0.9912
Epoch 87/400
570/570 [==============================] - 0s 2

In [ ]:
loss_value, accuracy_value = model.evaluate(test_data, test_labels)
print(f'Our accuracy was {accuracy_value}')

In [ ]:
def predictor(test_data, test_labels, index):
    predication = model.predict(test_data)
    if np.argmax(predication[index]) == test_labels[index]:
        print(f'This was correctly predicted to be a \"{test_labels[index]}\"!')
    else:
        print(f'This was incorrectly predicted to be a \"{np.argmax(predication[index])}\". It was actually a \"{test_labels[index]}\".')
        return predication


In [ ]:
predictor(test_data, test_labels, 149)